In [10]:
print(class_names)

['BrownSpot', 'Healthy', 'Hispa', 'LeafBlast']


In [1]:
import os

# Set the TensorFlow logging level to avoid informational messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2#
tf.keras.mixed_precision.set_global_policy('mixed_float16')#0 = all messages, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = filter out INFO, WARNING, and ERROR

# Use GPU if available
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Define the dataset paths
train_dir = "/Users/Mathi/Downloads/datasetfinal.zip/RiceDiseaseDataset/train"
validation_dir = "/Users/Mathi/Downloads/datasetfinal.zip/RiceDiseaseDataset/validation"

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Batch size
batch_size = 64  # Increase batch size if possible

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='categorical'
)

# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='categorical'
)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Adjust output size based on the number of classes
])

# Compile the model with a static learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,  # Reduced epochs for testing, increase if needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[reduce_lr]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('/Users/Mathi/Downloads/datasetfinal.zip/custom_trained_model.h5')

# Make predictions on custom images
# Use your local image path
image_path = "/Users/Mathi/Downloads/datasetfinal.zip/RiceDiseaseDataset/train/BrownSpot/IMG_2993.jpg"

# Load your own image
image = cv2.imread(image_path)
image = cv2.resize(image, (32, 32))
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=0)

# Make predictions
predictions = model.predict(image)
class_labels = train_generator.class_indices
predicted_class = list(class_labels.keys())[np.argmax(predictions)]

print("Predicted Class:", predicted_class)


Found 1600 images belonging to 4 classes.
Found 492 images belonging to 4 classes.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - accuracy: 0.2705 - loss: 2.5337 - val_accuracy: 0.2455 - val_loss: 1.3829 - learning_rate: 1.0000e-04
Epoch 2/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2955 - val_loss: 1.3618 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 105s 3s/step - accuracy: 0.4035 - loss: 1.7689 - val_accuracy: 0.3348 - val_loss: 1.4152 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.1818 - val_loss: 1.5090 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.3664 - loss: 1.7698 - val_accuracy: 0.3348 - val_loss: 1.4982 - learning_rate: 1.0000e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 1.5568 - learning_rate: 1.0000e-04
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.4175 - loss: 1.5988 - val_accuracy: 0.2902 - val_loss: 1.5771 - learning_rate: 1.0000e-04
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accu

Test accuracy: 0.4308035671710968
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Class: Hispa
